In [92]:
# do not touch this cell 
%load_ext jupyter_record


The jupyter_record extension is already loaded. To reload it, use:
  %reload_ext jupyter_record


# Imports


In [93]:
%%git_commit

import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack,vstack
import numpy as np


# Dataset Load




In [22]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv')
testing = pd.read_csv('../archive/Combined_Flights_2019.csv')


In [23]:
%%git_commit

training = training.dropna()
testing = testing.dropna()





In [24]:
%%git_commit

dropped_column = ['Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'DepDelayMinutes', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'DivAirportLandings']




In [25]:
%%git_commit

for column in dropped_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)


In [17]:
%%git_commit


training['CRSDepTime'].unique()

array([1025, 1555, 1630, ...,  438,  317,  441])

# Sort by Appropriate Data

'Month', 'DayofMonth', 'CRSDepTime' -> not actually completely information free, but good enough

In [26]:
%%git_commit
training = training.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])
testing = testing.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])


In [28]:
%%git_commit

training.keys()

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'CRSElapsedTime', 'Distance', 'Year', 'Quarter', 'Month',
       'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline',
       'IATA_Code_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'OriginAirportID', 'OriginAirportSeqID',
       'OriginCityMarketID', 'OriginCityName', 'OriginState',
       'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID',
       'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState',
       'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'CRSArrTime'],
      dtype='object')

# One Hot Encode


In [29]:
%%git_commit

d2_column = ['FlightDate', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners' ,  'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'OriginAirportSeqID', 'DestAirportSeqID', 'CRSArrTime', 'DestStateName', 'DestState', 'DestCityName', 'OriginStateName', 'OriginState', 'OriginCityName','Year',  'Diverted', 'Cancelled', 'Tail_Number', 'Quarter',  'DayOfWeek', ]


In [30]:
%%git_commit

for column in d2_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)
    

In [36]:
%%git_commit

training.reset_index()
testing.reset_index()


,index,Airline,Origin,Dest,CRSDepTime,CRSElapsedTime,Distance,Month,DayofMonth,Operating_Airline,OriginAirportID,OriginCityMarketID,OriginStateFips,OriginWac,DestAirportID,DestCityMarketID,DestStateFips,DestWac,DepDel15
0,7915905,United Air Lines Inc.,SFO,ORD,19,246.0,1846.0,1,1,UA,14771,32457,6,91,13930,30977,17,41,0.0
1,7916349,United Air Lines Inc.,SFO,IAH,25,215.0,1635.0,1,1,UA,14771,32457,6,91,12266,31453,48,74,0.0
2,7531120,Alaska Airlines Inc.,SEA,ORD,35,245.0,1721.0,1,1,AS,14747,30559,53,93,13930,30977,17,41,0.0
3,7915616,United Air Lines Inc.,LAX,EWR,45,318.0,2454.0,1,1,UA,12892,32575,6,91,11618,31703,34,21,0.0
4,7868252,Frontier Airlines Inc.,DEN,RSW,50,221.0,1607.0,1,1,F9,11292,30325,8,82,14635,31714,12,33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978587,4253635,JetBlue Airways,LAS,JFK,2347,292.0,2248.0,12,31,B6,12889,32211,32,85,12478,31703,36,22,0.0
978588,4311059,Delta Air Lines Inc.,LAS,ATL,2350,221.0,1747.0,12,31,DL,12889,32211,32,85,10397,30397,13,34,0.0
978589,4520800,United Air Lines Inc.,PHX,IAD,2355,240.0,1956.0,12,31,UA,14107,30466,4,81,12264,30852,51,38,0.0
978590,4253480,JetBlue Airways,JFK,BQN,2358,224.0,1576.0,12,31,B6,12478,31703,36,22,10732,30732,72,3,0.0


In [32]:
%%git_commit

encoders = {}

one_hot = ['DestWac', 'DestStateFips', 'DestCityMarketID', 'DestAirportID', 'OriginWac', 'OriginStateFips', 'OriginCityMarketID', 'OriginAirportID', 'Operating_Airline', 'Dest', 'Origin', 'Airline' ]


In [35]:
%%git_commit

for hot in one_hot:
    encode = OneHotEncoder(handle_unknown = 'ignore')
    encode.fit(training[hot].to_numpy().reshape(-1, 1))
    encoders[hot] = encode

# Generate Time Series

In [66]:
%%git_commit

def get_previous_rows(df, current_row_index, previous_rows_num = 10):
    ''' Given a sorted pandas dataframe and row index, return previous rows
    
    df: dataframe, eg. the training dataset
    current_row_index: row index to get previous rows from
    previous_rows_num: 
    
    
    '''
    if previous_rows_num <= 0:
        raise ValueError('invalid row number')
    index = current_row_index - previous_rows_num
    if index < 0:
        raise ValueError('current_row_index less than previous_rows_num')
    subset = df.iloc[index:current_row_index]
    return subset


month_days = {1: 31, 2: 29, 3: 31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}


def _get_previous_days(num, month, day):
    last = (month, day)
    previous_days = [last]
    for i in range(num - 1):
        m, d = previous_days[-1]
        if d == 1:
            m -= 1
            if m == 0:
                raise ValueError('previous days not in dataset')
            d = month_days[m]
        else:
            d -= 1
        
        previous_days.append((d, m))



def get_previous_days(df, current_row_index, previous_days = 14):
    ''' Given a sorted pandas dataframe with columns 'Month', 'DayofMonth', 'CRSDepTime', and a row index, 
    return the rows that are 'previous_days' days before 'df.iloc[current_row_index]'
    
    Note: this includes flights on the current day that have an estimated departure time BEFORE the df.iloc[current_row_index]
    
    df: dataframe, eg. training dataset
    current_row_index: row index to evaluate
    previous_days: number of days prior to row, counting current one
    
    '''
    raise NotImplemented()
    



In [61]:
%%git_commit



# Model Training



In [40]:
%%git_commit

d3_column = ['Month', 'DayofMonth']


for column in d3_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)


In [86]:
%%git_commit

# features: count of previous 20
not_hot = ['CRSDepTime', 'CRSElapsedTime', 'Distance']



def get_feature_vector(sub_df, is_scalar = False):
    array = []
    for hot in one_hot:
        x = np.asarray(sub_df[hot]).reshape(-1, 1)
        x = encoders[hot].transform(x)
        array.append(x)
    for col in not_hot:
        x = np.asarray(sub_df[col]).reshape(-1, 1)
        array.append(x)

    X_train = hstack(array)
    y_train = np.asarray(sub_df['DepDel15']).reshape(-1)
    
    return X_train, y_train


def get_features_row(df, index):
    prev_rows = get_previous_rows(df, index)
    prev_X, prev_Y = get_feature_vector(prev_rows)
    prev_Y = prev_Y.reshape(-1, 1)   
    prev_features = hstack([prev_X, prev_Y])
    prev_features = np.sum(prev_features, axis = 0)
    X, y = get_feature_vector(df.iloc[index], is_scalar = True)
    X = hstack([prev_features, X])
    return X, y


get_features_row(training, 20)
    

(<1x4873 sparse matrix of type '<class 'numpy.float64'>'
 	with 116 stored elements in COOrdinate format>,
 array([0.]))

In [96]:
%%git_commit

X_train = []
y_train = []
X_test = []
y_test = []

print(training.shape)
print(testing.shape)

(980430, 16)
(978592, 16)


In [97]:
%%git_commit

train = np.random.choice(np.arange(20, 980430), size = 50000, replace=False)
test =  np.random.choice(np.arange(20, 978592), size = 50000, replace=False)  

In [98]:
%%git_commit

for i in train:
    get_features_row(training, 20)


885243
848541
54361
16506
517589
358065
632072
729437
702387
239871
372434
209758
68965
130382
473897
938115
807234
855237
565889
742052
812797
12885
265695
656599
749737
262642
328657
776046
34454
217761
477394
438934
231776
391088
798075
242838
467664
812188
871573
378034
64479
879440
88933
392355
277693
944101
739746
106041
504496
707057
54198
187846
13304
496992
16491
115132
295510
709010
127898
109055
612161
306541
397333
810081
267953
822224
379574
888432
846964
907555
936503
104307
373505
125945
151522
454134
364980
484076
700515
364327
600951
401439
535459
634815
287474
520096
366089
428730
357998
606234
320019
583707
519395
146360
17271
294491
865835
241694
658583
639992
209507
858069
398649
668136
510780
905301
274678
96735
924326
810026
615989
765435
587585
447162
106585
182384
845661
114483
966453
642966
93313
103726
885193
435036
553405
371536
468427
494271
502042
311372
396363
817423
136703
154878
268881
521892
274487
837796
263423
577991
358321
864794
488153
460727
14429

845169
645235
117947
397423
569531
281442
452424
844933
604658
847502
938556
123099
305160
838827
485606
592697
57640
667971
88611
518955
881315
907657
570129
800302
328814
194725
656918
837976
820119
528030
120160
591300
243694
394338
278365
790927
719498
271514
307143
834973
736541
234687
931332
641598
507052
35099
621892
80786
246371
841530
660088
400986
29765
246470
969773
32640
430632
498146
533621
827577
288410
583925
804667
747614
638708
66278
659206
204537
475771
493765
780111
800116
731517
786514
73526
148331
445257
482388
125628
736915
645343
908641
248598
329971
580089
928169
552415
169203
163456
522668
646196
509645
180157
279625
578822
401293
489869
894604
236202
449069
154813
360353
438190
635734
669773
230304
615083
425495
557542
101340
50241
974391
262526
132729
60308
845558
659346
895964
927631
374105
309340
277634
182844
723337
103553
339090
510238
58440
260038
813031
831618
704631
40241
640664
500183
835416
970535
791893
478986
592150
779875
284328
276245
629428
4238

444665
27130
556039
918192
112018
601104
344667
772175
813057
815944
529820
634873
525009
384240
81583
31800
70169
360863
925056
923681
635619
888786
557359
687197
91326
455823
305194
308557
60199
507442
445739
666429
463823
390125
443383
538026
444483
492169
255723
256801
881496
249711
874693
162998
959453
357413
888721
700848
110909
868363
345540
229762
877161
31118
773310
705507
474837
166187
311059
576599
236148
806963
175086
185401
698847
55162
822851
431475
824853
275634
146943
242226
690282
730666
187639
93586
227014
970284
353891
375772
739179
56680
899128
793947
22047
255016
439068
425353
266809
69169
618816
47336
587096
546669
123370
549930
938172
637906
715101
681769
823936
404782
247037
911667
567094
833765
760379
4136
226975
623436
891022
434457
436176
19322
251965
749096
666609
241903
928986
842295
699333
851791
642109
119785
912703
460635
502783
799413
611906
205693
98294
950633
710361
698245
546858
302318
131396
244350
918061
498986
17918
504592
590342
846102
141585
929

904475
194087
1144
946689
815236
708023
357071
853315
838700
959589
198642
733078
547480
147235
896089
808450
859744
131560
766263
33390
134545
218254
31310
188368
99428
859659
27478
255782
582161
405237
747615
731240
841950
640254
407663
628230
501136
294486
360900
97794
797478
529944
570606
359674
185787
63157
8370
830415
825857
261457
644847
267165
627987
67638
639343
111070
147780
558268
841057
919278
634025
749773
484138
665326
309061
495243
363129
898607
330337
96851
941035
517012
342572
229766
464351
952068
630759
358516
596165
192322
414347
234498
205248
37701
696393
39704
318390
939410
846510
928545
247172
912562
373827
530853
629332
382519
229666
876990
885098
289177
462724
376266
423738
416218
214654
250623
67766
384419
213238
137208
970866
807648
297902
928410
803439
292727
848509
194786
547361
686951
285861
696218
345985
421649
485225
782476
82762
550587
200978
925445
20246
623066
13525
855878
515129
253451
165118
745023
418623
765639
591980
289858
672631
582422
868952
511

795009
754858
682906
685026
965184
167833
836132
215890
621139
469555
362318
287250
378696
71679
786075
629112
935584
580400
932518
908777
907519
969337
107570
414493
766752
640527
492943
141162
252973
551692
901698
802341
340962
494642
838502
835179
705258
850195
146184
232343
907511
428232
230365
743768
503380
303126
759892
709023
128761
248891
755278
343729
372636
365515
643182
386147
507175
182696
152795
48364
915817
438335
113369
417248
743764
12601
940177
232478
82209
819339
53247
440585
341370
606749
840539
950372
903717
475907
634314
112550
473279
928819
516514
133676
111096
859673
527698
737399
436218
164688
720166
556121
198198
423842
271338
454196
486065
943374
273669
709782
390767
30704
579525
18607
486196
508914
688065
818580
597331
620665
826279
98548
693865
491389
614910
405184
882554
113125
233377
177826
801696
238651
380469
387033
382116
56871
705366
784042
456826
925757
579182
559773
397314
485065
959015
783101
409383
637394
166054
644939
91743
387714
122648
50139
968

...

637668
544288
141474
246538
439215
440494
338429
141259
764270
493071
44014
820173
478492
924335
593476
419339
745065
38142
242146
286366
744750
826177
36642
425476
912867
459651
307642
528297
810886
92496
941497
54231
34494
933439
730082
313172
639057
641267
153589
62238
649891
452889
857166
364954
902469
33506
526762
289772
795197
591291
846068
205652
659060
893343
949646
467728
378119
592382
172413
632663
138283
694629
555061
195027
153668
14500
453777
515374
777035
748206
214289
760467
35777
854040
51689
865028
497222
815087
652205
630962
466742
509225
420666
700705
193985
203208
802117
40580
394990
722599
523444
798045
348560
55063
108612
737794
76982
284428
373517
58321
171103
719242
710242
932020
673817
706900
409878
63671
697928
197812
509099
955348
313808
393037
351082
497368
641698
500994
581657
441089
700206
595008
613336
554833
710700
450800
880777
547733
709941
89048
762546
628670
128908
498587
92851
838824
710149
363708
585730
147641
395873
412917
200677
575950
317971
284

376948
518608
878390
483677
794795
663186
444718
561956
102273
496536
283460
351192
25259
818233
742583
97605
154280
217095
192140
56538
762913
447940
115332
180643
515031
28073
593048
918856
112323
760777
239812
546964
960016
371990
211127
327776
416109
887150
523420
55411
141932
965191
546193
459300
865498
836324
317663
806578
486596
190019
201962
773355
710575
262401
685141
112030
120729
934525
147271
151033
674196
55652
622152
669719
944400
371791
532354
489646
223233
304920
93179
972847
698654
358176
328855
451417
829214
122718
212941
43579
388934
850676
1055
12776
192627
570972
385865
515142
378219
963685
653734
65149
765522
141838
260082
118561
505810
44340
395056
454422
564031
521523
476575
720796
225910
625659
96619
231540
251407
405661
675194
312691
632670
821109
922469
335438
761364
485632
52805
380561
789524
713292
784904
65420
811211
327934
319270
181959
758584
357109
879096
600047
915524
939843
472803
948373
643996
89762
580208
55
850886
200664
941760
192886
532426
642047

391984
706676
123530
533676
13429
818705
387137
402424
950147
779215
420257
881872
366752
87968
362820
66915
702036
226289
281895
974063
945154
399335
831616
836774
274463
226315
98094
256615
218048
46489
17871
881744
360843
947231
434763
269924
689886
215
170836
929176
182789
245924
127083
790229
837152
848028
608780
862242
11821
686695
967839
590983
852438
27311
859511
938538
944614
818961
440928
957936
590056
951638
892734
477117
562971
941514
213797
653006
231507
704344
620510
670224
363281
214038
332188
281651
552435
517085
687951
299340
10442
12180
922973
720292
344541
741986
173172
386584
410312
846035
127118
709869
847726
165028
78100
514596
432050
175849
75531
587453
359998
379077
717859
199022
178670
722127
976795
879128
502077
833733
719780
944733
46349
716164
583372
510847
576921
358359
240658
838393
911617
649771
17462
220077
692932
674265
508812
490028
618051
445738
301195
56420
408026
60545
316276
877844
673414
982
456585
547880
970726
128593
308156
556481
203400
400007


354586
200499
4455
501949
703409
630270
856457
167946
40155
632685
786809
412052
773992
298015
852689
929881
146813
124856
261423
9936
413254
383689
350463
251591
976363
178801
722686
379511
272705
368439
99313
199865
645681
722344
471707
789100
534511
89553
79685
865022
420198
317258
205663
130381
572299
730597
905470
966029
116365
398424
295720
460630
783318
909097
954865
553101
675421
209173
470141
229181
242601
458238
687666
583242
743747
77264
222468
580610
771761
971901
87939
959221
49864
282913
335183
242413
327737
705193
301320
832116
448751
883352
729700
392693
523523
850548
71183
13584
735032
845414
827400
452783
870395
16171
661487
964426
106123
896971
408436
32727
812018
738061
814833
905119
24827
207265
103851
363610
865839
602873
750217
716283
284395
130334
466290
453892
429610
237713
156605
248519
260606
505116
556786
842432
773092
321075
530048
485059
884728
173961
960753
948209
540998
154602
347690
69231
826337
907360
181696
930399
154911
689038
663275
351699
805147
53

274369
106937
530285
384589
436054
193212
29989
536951
592386
936068
396507
914295
474834
793763
373908
141398
126880
941887
670415
142569
30269
403979
81340
63867
895524
517193
748973
612218
767067
870856
929540
781495
892521
451005
806087
832966
29802
80101
625313
101123
173611
153054
401618
143902
394174
664174
895968
210445
102597
879888
592336
974758
937670
689162
791519
784351
478290
651036
750799
513706
718508
534148
976043
939388
956203
958837
303921
145753
339373
905209
712873
871548
100337
633811
396590
843235
820778
925548
683409
483289
437639
690658
498183
417358
230170
628405
734305
19452
403824
683438
643524
54341
339672
603857
201629
354924
11991
390793
250426
929299
635719
161775
787914
477244
444170
491900
452546
631417
587329
819624
915708
306210
644773
143650
359371
538883
853477
901561
685741
810846
590220
685365
756409
875543
762572
796294
834995
101263
683721
393657
206202
826134
407305
800794
356891
432995
243306
252416
485675
56081
165211
216636
785226
869683
96

In [ ]:
%%git_commit
